In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns  
import re
import os

In [6]:
# フォルダのパス
folder_path = "/Users/aruhy/OneDrive/デスクトップ/data/24-3-21"

# フォルダ内のファイルのリストを取得
file_list = os.listdir(folder_path)

# ファイルごとに処理を行う
df = pd.DataFrame(columns=['V_cell','P_b','T_cell','stoichi_anode','stoichi_cathode','W_ch','D_ch','i_ave'])
for file_name in file_list:
     # ファイルの絶対パスを作成
    file_path = os.path.join(folder_path, file_name)
    
    # ファイルがディレクトリでない場合に処理を行う
    if not os.path.isdir(file_path):
        # CSVファイルのパス
        with open(file_path, 'r') as file:
            content = file.read()
            
        # 文字列の置換
        old_string = 'cd.IlMag (A/m^2) '
        new_string = '\n%'
        updated_content = content.replace(old_string, new_string)
        # ファイルを書き込みモードで開いて、更新された内容を書き込む
        with open(file_path, 'w') as file:
            file.write(updated_content)
            
        with open(file_path, 'r') as file:
            lines = file.readlines()
        # データを保持するためのリストを初期化
        data = {
            'V_cell': [],
            'P_b': [],
            'T_cell': [],
            'stoichi_anode': [],
            'stoichi_cathode': []
        }

        # データを抽出してリストに追加
        for line in lines:
            if line.startswith('%@'): #'%@'で始まる行だけTrue、つまり処理が行われる
                parts = line.split(':')
                #index = int(parts[0].split('@')[1].strip())
                #data['Index'].append(index)
                values = parts[1].strip().split(',')
                data['V_cell'].append(float(values[0].split('=')[1].strip().split(' ')[0]))
                data['P_b'].append(float(values[1].split('=')[1].strip().split(' ')[0]))
                data['T_cell'].append(float(values[2].split('=')[1].strip().split(' ')[0]))
                data['stoichi_anode'].append(float(values[3].split('=')[1].strip()))
                data['stoichi_cathode'].append(float(values[4].split('=')[1].strip().split(' ')[0]))

        # データフレームに変換
        df_param = pd.DataFrame(data)

        #W_chとD_chのカラムの追加
        W_ch = re.findall(r'W(\d\.\d+)_',file_path) # ファイル名から数字を抽出
        D_ch = re.findall(r'D(\d\.\d+)',file_path) # ファイル名から数字を抽出

        # 数字を一列のリストに格納
        data_W = W_ch * 117
        data_D = D_ch * 117

        # 117行のデータフレームを作成
        df_param2 = pd.DataFrame({'W_ch':data_W,'D_ch':data_D})

        #電流密度のデータフレームを作製
        current_row = 1
        split_pattern = re.compile(r'\s+')  # 空白文字に基づいて行を分割する正規表現パターン
        for line in lines:
            if current_row == 126:
                i_data = split_pattern.split(line.strip())
                df_i = pd.DataFrame({'i_ave':i_data})
                break #ループを抜ける
            current_row += 1

        #データフレームの結合
        df_k = pd.concat([df_param,df_param2], axis=1)
        df_k = pd.concat([df_k,df_i], axis=1)
        df = pd.concat([df,df_k], axis=0)    
# データフレームを表示
df


C:\Users\aruhy\AppData\Local\Temp\ipykernel_22380\4167953135.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,df_k], axis=0)


,V_cell,P_b,T_cell,stoichi_anode,stoichi_cathode,W_ch,D_ch,i_ave
0,1.00,101330.0,373.15,1.5,2.0,0.4,0.4,10.380173171667092
1,0.95,101330.0,373.15,1.5,2.0,0.4,0.4,44.93078986452143
2,0.90,101330.0,373.15,1.5,2.0,0.4,0.4,159.31548422329277
3,0.85,101330.0,373.15,1.5,2.0,0.4,0.4,413.51944602456126
4,0.80,101330.0,373.15,1.5,2.0,0.4,0.4,819.6215377516025
...,...,...,...,...,...,...,...,...
112,0.60,101330.0,393.15,5.5,6.0,1.6,1.6,5128.186267520728
113,0.55,101330.0,393.15,5.5,6.0,1.6,1.6,6178.536866281978
114,0.50,101330.0,393.15,5.5,6.0,1.6,1.6,7269.896159764467
115,0.45,101330.0,393.15,5.5,6.0,1.6,1.6,8393.22149589775


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1053 entries, 0 to 116
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   V_cell           1053 non-null   float64
 1   P_b              1053 non-null   float64
 2   T_cell           1053 non-null   float64
 3   stoichi_anode    1053 non-null   float64
 4   stoichi_cathode  1053 non-null   float64
 5   W_ch             1053 non-null   object 
 6   D_ch             1053 non-null   object 
 7   i_ave            1053 non-null   object 
dtypes: float64(5), object(3)
memory usage: 74.0+ KB


In [8]:
df= df.astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1053 entries, 0 to 116
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   V_cell           1053 non-null   float64
 1   P_b              1053 non-null   float64
 2   T_cell           1053 non-null   float64
 3   stoichi_anode    1053 non-null   float64
 4   stoichi_cathode  1053 non-null   float64
 5   W_ch             1053 non-null   float64
 6   D_ch             1053 non-null   float64
 7   i_ave            1053 non-null   float64
dtypes: float64(8)
memory usage: 74.0 KB


In [9]:
df['W']=df['V_cell']*df['i_ave']
df

,V_cell,P_b,T_cell,stoichi_anode,stoichi_cathode,W_ch,D_ch,i_ave,W
0,1.00,101330.0,373.15,1.5,2.0,0.4,0.4,10.380173,10.380173
1,0.95,101330.0,373.15,1.5,2.0,0.4,0.4,44.930790,42.684250
2,0.90,101330.0,373.15,1.5,2.0,0.4,0.4,159.315484,143.383936
3,0.85,101330.0,373.15,1.5,2.0,0.4,0.4,413.519446,351.491529
4,0.80,101330.0,373.15,1.5,2.0,0.4,0.4,819.621538,655.697230
...,...,...,...,...,...,...,...,...,...
112,0.60,101330.0,393.15,5.5,6.0,1.6,1.6,5128.186268,3076.911761
113,0.55,101330.0,393.15,5.5,6.0,1.6,1.6,6178.536866,3398.195276
114,0.50,101330.0,393.15,5.5,6.0,1.6,1.6,7269.896160,3634.948080
115,0.45,101330.0,393.15,5.5,6.0,1.6,1.6,8393.221496,3776.949673


In [10]:
df2 = df[(df['V_cell']==0.60)]
df2

,V_cell,P_b,T_cell,stoichi_anode,stoichi_cathode,W_ch,D_ch,i_ave,W
8,0.6,101330.0,373.15,1.5,2.0,0.4,0.4,3591.054698,2154.632819
21,0.6,101330.0,373.15,1.5,4.0,0.4,0.4,3594.562527,2156.737516
34,0.6,101330.0,373.15,1.5,6.0,0.4,0.4,3596.141769,2157.685062
47,0.6,101330.0,383.15,3.5,2.0,0.4,0.4,4371.434030,2622.860418
60,0.6,101330.0,383.15,3.5,4.0,0.4,0.4,4376.898529,2626.139118
...,...,...,...,...,...,...,...,...,...
60,0.6,101330.0,383.15,3.5,4.0,1.6,1.6,4281.831057,2569.098634
73,0.6,101330.0,383.15,3.5,6.0,1.6,1.6,4284.766318,2570.859791
86,0.6,101330.0,393.15,5.5,2.0,1.6,1.6,5114.359982,3068.615989
99,0.6,101330.0,393.15,5.5,4.0,1.6,1.6,5123.909012,3074.345407


In [13]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [28]:
# 分散分析モデルの定義
model = ols('W ~ C(P_b) + C(T_cell) + C(stoichi_anode) + C(stoichi_cathode) + C(W_ch) + C(D_ch)', data=df2).fit()

# 分散分析表の表示
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
C(P_b),1.149920e+06,2.0,3711.330491,3.729222e-70
C(T_cell),1.417920e+07,2.0,45762.909253,3.998098e-107
C(stoichi_anode),7.939560e+03,2.0,25.624678,5.079228e-09
C(stoichi_cathode),3.204527e+02,2.0,1.034251,3.610162e-01
C(W_ch),2.858759e+04,2.0,92.265540,4.232515e-20
C(D_ch),1.241695e+03,2.0,4.007530,2.263160e-02
Residual,1.053457e+04,68.0,NaN,NaN


In [29]:
# 各要素の変動を取得
total_variance = anova_table['sum_sq']['Residual']  + anova_table['sum_sq']['C(P_b)'] + \
                anova_table['sum_sq']['C(T_cell)'] + anova_table['sum_sq']['C(stoichi_anode)'] + anova_table['sum_sq']['C(stoichi_cathode)'] + \
                anova_table['sum_sq']['C(W_ch)'] + anova_table['sum_sq']['C(D_ch)']

treatment_variance = anova_table['sum_sq']['C(P_b)'] + \
                    anova_table['sum_sq']['C(T_cell)'] + anova_table['sum_sq']['C(stoichi_anode)'] + \
                    anova_table['sum_sq']['C(stoichi_cathode)'] + anova_table['sum_sq']['C(W_ch)'] + \
                    anova_table['sum_sq']['C(D_ch)']

# 各因子の寄与率を計算してデータフレームに追加
anova_table['Contribution'] = (anova_table['sum_sq']-anova_table['df']*anova_table['sum_sq']['Residual']/anova_table['df']['Residual'])/ anova_table['sum_sq'].sum()

anova_table

,sum_sq,df,F,PR(>F),Contribution
C(P_b),1.149920e+06,2.0,3711.330491,3.729222e-70,7.475807e-02
C(T_cell),1.417920e+07,2.0,45762.909253,3.998098e-107,9.220397e-01
C(stoichi_anode),7.939560e+03,2.0,25.624678,5.079228e-09,4.961535e-04
C(stoichi_cathode),3.204527e+02,2.0,1.034251,3.610162e-01,6.901098e-07
C(W_ch),2.858759e+04,2.0,92.265540,4.232515e-20,1.838875e-03
C(D_ch),1.241695e+03,2.0,4.007530,2.263160e-02,6.059760e-05
Residual,1.053457e+04,68.0,NaN,NaN,0.000000e+00


In [30]:
anova_table['sum_sq']['Residual']

10534.572838455277

In [24]:
(anova_table['sum_sq']['C(T_cell)']-(2*anova_table['sum_sq']['Residual']/1028))/total_variance

0.7695584303035474

In [25]:
(anova_table['sum_sq']['C(P_b)']-(2*anova_table['sum_sq']['Residual']/1028))/total_variance

0.0662716229694886

In [17]:
anova_table['sum_sq']['C(P_b)']/total_variance

0.07477821620322962

In [31]:
from sklearn.preprocessing import StandardScaler

In [33]:
df.to_csv("CD_OA")

In [34]:
for col in df.columns:
    scaler =  StandardScaler()
    df[col] = scaler.fit_transform(np.array(df[col].values).reshape(-1, 1))
    
df.head()

,V_cell,P_b,T_cell,stoichi_anode,stoichi_cathode,W_ch,D_ch,i_ave,W
0,1.603567,-1.224725,-1.224745,-1.224745,-1.224745,-1.069045,-1.069045,-1.158819,-1.427550
1,1.336306,-1.224725,-1.224745,-1.224745,-1.224745,-1.069045,-1.069045,-1.147149,-1.402862
2,1.069045,-1.224725,-1.224745,-1.224745,-1.224745,-1.069045,-1.069045,-1.108512,-1.325902
3,0.801784,-1.224725,-1.224745,-1.224745,-1.224745,-1.069045,-1.069045,-1.022649,-1.166857
4,0.534522,-1.224725,-1.224745,-1.224745,-1.224745,-1.069045,-1.069045,-0.885477,-0.934369


In [ ]:
df.to_csv("CD_OA_std")